# 1. Read Dataset(Target Stories)

In [30]:
import glob
file_names = glob.glob("/Users/ibrahim/Desktop/Preprocess/query/*.txt")
raw_documents=[]
for file in file_names:
    try:
        with open(file, "r", encoding="utf-8") as f: raw_documents.append(f.read())
    except:
        pass
print("Number of documents: ",len(raw_documents))

Number of documents:  5


##  1.1 Read Dataset(Query Story)

In [31]:
querydoc=glob.glob("/Users/ibrahim/Desktop/Preprocess/AllStories/*.txt")
query_documents=[]
for file in querydoc:
    try:
        with open(file, "r", encoding="utf-8") as f: query_documents.append(f.read())
    except:
        pass
query = query_documents[0].lower()
#print("Number of documents: ",len(query_documents))

# 1.1 Locality Sensitive Hashing (LSH) Algorithm (First Layer)
https://nbviewer.org/github/bassimeledath/quora_profile/blob/master/questions_analysis.ipynb

https://www.pinecone.io/learn/locality-sensitive-hashing/

In [32]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest

In [33]:
#Preprocess will split a string of text into individual tokens/shingles based on whitespace.
def preprocess(text):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    tokens = text
    # Removing punctuations in string
    # Using loop + punctuation string
    for ele in tokens:
        if ele in punc:
            tokens = tokens.replace(ele, "")
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [34]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    #print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [35]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    result={}
    result = database.iloc[idx_array]
    
    #print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [36]:
def LSH_Candiadates_List(candidates_list):
    db = pd.DataFrame(candidates_list)#We have 67 Sentences
    db=db[0]
    #print(db)
    #Number of Permutations
    permutations = 800
    #Number of Recommendations to return
    num_recommendations = 5
    forest = get_forest(db, permutations)
    num_recommendations = 10
    #query = query_documents[0].lower()
    result = predict(query, db, permutations, num_recommendations, forest)
    #print('\n Top similar sentences \n', result1)


    #Create a list of candiates to be taken to the next layer which is Cosine Simialrity
    candidates=[]
    candidates=result.values
    return candidates

# Print a the candidates list from LSH

In [37]:
LSH_Candidates= LSH_Candiadates_List(raw_documents)

# 1.2 Cosine Similarity (The second layer)

https://towardsdatascience.com/how-to-rank-text-content-by-semantic-similarity-4d2419a84c32

In [38]:
from re import sub
from gensim.utils import simple_preprocess

#query_string = 'fruit and vegetables'
#documents = ['cars drive on the road', 'tomatoes are actually fruit']

stopwords = []

# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in LSH_Candidates]
#print(corpus)
query = preprocess(query)
#print (query)

In [39]:
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")    
similarity_index = WordEmbeddingSimilarityIndex(glove)

# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

100%|██████████████████████████████████████| 1045/1045 [00:08<00:00, 128.45it/s]


In [40]:
# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def Cosine_Candiadates_List(candidates_list) :
    
    query_tf = tfidf[dictionary.doc2bow(query)]

    index = SoftCosineSimilarity(
                tfidf[[dictionary.doc2bow(document) for document in corpus]],
                similarity_matrix)

    doc_similarity_scores = index[query_tf] 


# Output the sorted similarity scores and documents
    NW_Candiadates=[]
    sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
    for idx in sorted_indexes:
        if doc_similarity_scores[idx]>0.3:
            print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {LSH_Candidates[idx]}')
            NW_Candiadates.append(LSH_Candidates[idx])
    return NW_Candiadates

In [41]:
Pre_NW_Candiadates= Cosine_Candiadates_List(LSH_Candidates)

1 	 0.622 	 When it was the third night, Dunyazad asked her sister to finish the story. ‘With pleasure,’ said Shahrazad and went on: ‘I have heard, O fortunate king, that the third old man told the ‘ifrit a more remarkable story than the other two, and that in his astonishment and delight the ‘ifrit granted him the remaining share of the blood debt and allowed the merchant to go free. For his part, the merchant went and thanked the old men, who congratulated him on his safety, after which each of them went home. This, however, is not more surprising than the tale of the fisherman.’ When the king asked what that was, she went on:

I have heard, O fortunate king, that there once was a poor, elderly fisherman with a wife and three children, who was in the habit of casting his net exactly four times each day. He went out to the shore at noon one day, put down his basket, tucked up his shirt, waded into the sea and cast his net. He waited until it had sunk down before pulling its cords toge

In [42]:
NW_Candiadates=[]
NW_Candiadates=Pre_NW_Candiadates[0]#Take the story that has the maximum cosine similarity
#print(NW_Candiadates)

# Send the target Story to be processed again
1. Remove punctuations
3. Lemmatizations(optional)
4. Tokenization each 3 sentences together


In [43]:
import nltk 
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

def Before_NW_Processing(candidates_list):
    paragraphs = list(filter(lambda x : x != '', candidates_list.split('.')))#https://stackoverflow.com/questions/53240763/python-how-to-separate-paragraphs-from-text
    len(paragraphs)
    paragraphs2=[]
    for i in paragraphs:
        i = i.lower()
        paragraphs2.append(i)
        import re
 
    # Removing punctuations in string
    # Using loop + punctuation string
    final_paragraphs_list=[]
    for ele in paragraphs2:
        x = re.sub(r'[^\w\s]', '', ele)
        x = x.replace('\n'," ")
        final_paragraphs_list.append(x)
   
    #https://stackoverflow.com/questions/50685343/how-to-lemmatize-a-list-of-sentences

    lmtzr = WordNetLemmatizer()
    lemmatized = [[lmtzr.lemmatize(word) for word in word_tokenize(s)]
                  for s in final_paragraphs_list]
#print(lemmatized[2])
#[['i', 'like', 'car'], ['cat', 'are', 'the', 'best']]

    Final_Lemmatized=[]
    for i in range(len(final_paragraphs_list)):
        Final_Lemmatized.append(" ".join(word for word in lemmatized[i]))
    return Final_Lemmatized

In [45]:

Final_Target_Candidates=[]
Final_Target_Candidates=Before_NW_Processing(NW_Candiadates)
#len(Final_Target_Candidates)

query = query_documents[0].lower()
Final_Query_Candidates=[]
Final_Query_Candidates= Before_NW_Processing(query)
#len(Final_Query_Candidates)

# Tokenization each 3 sentences together


In [46]:
def Three_Sentences_Tokenization(candidates_list):
    Final_Lemmatized=candidates_list
    Final_Dataset=[]
    i=0
    if len(Final_Lemmatized) % 3==0:
        while(i < len(Final_Lemmatized)):
            #if len(x) < len(test):
            if Final_Lemmatized[i]==Final_Lemmatized[-2]:
                Final_Dataset.append(Final_Lemmatized[-2]+" "+Final_Lemmatized[-1])
                break
            else:
                Final_Dataset.append(Final_Lemmatized[i]+" "+Final_Lemmatized[i+1]+" "+Final_Lemmatized[i+2])
            i += 3
        return Final_Dataset
    else:
        while(i < len(Final_Lemmatized)):
            #if len(x) < len(test):
            if Final_Lemmatized[i]==Final_Lemmatized[-1]:
                Final_Dataset.append(Final_Lemmatized[i])
                Final_Dataset[-1]=Final_Dataset[-2]+" "+Final_Dataset[-1]
                Final_Dataset.remove(Final_Dataset[-2])
                break
            elif Final_Lemmatized[i]==Final_Lemmatized[-2]:
                Final_Dataset.append(Final_Lemmatized[i]+" "+Final_Lemmatized[i+1])
                #x.remove(x[-2])
                break
            else:
                Final_Dataset.append(Final_Lemmatized[i]+" "+Final_Lemmatized[i+1]+" "+Final_Lemmatized[i+2])
            i += 3
        return Final_Dataset
        
#print(Final_Query)

In [47]:
Final_Target=[]
Final_Target= Three_Sentences_Tokenization(Final_Target_Candidates)
Final_Query=[]
Final_Query= Three_Sentences_Tokenization(Final_Query_Candidates)

# Needlman-Wunch 

In [50]:
from minineedle import needle, core
def needlman_wunch(Target_candidates_list,Query_list):
    
    
    R = {} # Dicitonary to Save query, target sentence, precent identity and the actual aligmnet
    IR=0 #IndexRange

    query=Query_list[IR].split()
    if IR==0:
        for i in Target_candidates_list[IR:IR+1]:
        #print("t--------------",i)
            alignment = needle.NeedlemanWunsch(query, i.split())
            x = alignment.get_identity()
            y = alignment 
            R[i]=[x , query, i, y]
        max_value = max(R.values())
        print('Query: ',Query_list[IR],'\n')
        print('Target Sentence: \n',max_value[2],'\n')
        print('Perecent Identity using NW:',max_value[0])
    #print('\n',max_value[3]) 
    else:
        for i in Target_candidates_list[IR-1:IR+1]:
        #print("t--------------",i)
            alignment = needle.NeedlemanWunsch(query, i.split())
            x = alignment.get_identity()
            y = alignment 
            R[i]=[x , query, i, y]
        max_value = max(R.values())
        print('Query: ',Query_list[IR],'\n')
        print('Target Sentence: \n',max_value[2],'\n')
        print('Perecent Identity using NW:',max_value[0]) 

# Call the Needlman-Wunch with Final_Target and Final Final_Query Datasets

In [51]:
needlman_wunch(Final_Target,Final_Query)

Query:  when it wa the third night and the king had had his will of the wazirs daughter dunyazad her sister said to her finish for u that tale of thine and she replied with joy and goodly gree it hath reached me o auspicious king that when the third old man told a tale to the jinni more wondrous than the two preceding the jinni marvelled with exceeding marvel and shaking with delight cried lo i have given thee the remainder of the merchant punishment and for thy sake have i released him thereupon the merchant embraced the old men and thanked them and these shaykhs wished him joy on being saved and fared forth each one for his own city yet this tale is not more wondrous than the fisherman story 

Target Sentence: 
 when it wa the third night dunyazad asked her sister to finish the story with pleasure said shahrazad and went on i have heard o fortunate king that the third old man told the ifrit a more remarkable story than the other two and that in his astonishment and delight the ifrit 